In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime

# ================================
# تارگت درست و واقعی
# ================================
TARGET_COL = 'AssetID_8343'                                      # درست!
INPUT_FILE  = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_FILE = r"C:\BI\regression_output_for_assetid_8343.xlsx"  # اسم معنی‌دار و حرفه‌ای
LOG_FILE    = r"C:\BI\regression_pressure_8343_log.txt"

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log(f"شروع پیش‌بینی فشار پمپ اصلی روغن → {TARGET_COL}")

# خواندن فایل
df_original = pd.read_excel(INPUT_FILE)
log(f"فایل خوانده شد → {len(df_original):,} ردیف")

# چک نهایی — دیگه خطا نمی‌ده
if TARGET_COL not in df_original.columns:
    log(f"خطا: ستون {TARGET_COL} وجود ندارد!")
    exit()
else:
    log(f"موفقیت: ستون {TARGET_COL} پیدا شد (فشار پمپ اصلی روغن)")

# حذف ستون‌های غیرعددی برای مدل
non_features = ['id', 'RecordDate', 'RecordTime']
model_df = df_original.drop(columns=[c for c in non_features if c in df_original.columns], errors='ignore')

X = model_df.drop(TARGET_COL, axis=1)
y = model_df[TARGET_COL]

# نرمالایز
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# آموزش
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=1200, learning_rate=0.02, max_depth=8,
                     subsample=0.9, colsample_bytree=0.8, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# ارزیابی
y_pred_test = model.predict(X_test)
mae  = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2   = r2_score(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-8))) * 100

log("="*60)
log(f"نتایج نهایی پیش‌بینی فشار پمپ اصلی روغن ({TARGET_COL})")
log(f"MAE   : {mae:.6f} bar")
log(f"RMSE  : {rmse:.6f} bar")
log(f"R²    : {r2:.6f}")
log(f"MAPE  : {mape:.3f}%")
log("="*60)

# پیش‌بینی کل داده
y_pred_full = model.predict(X_scaled_df)

# خروجی نهایی
result = df_original.copy()
result[f'{TARGET_COL}_Predicted'] = y_pred_full
result['Error (bar)'] = result[TARGET_COL] - y_pred_full
result['Absolute_Error (bar)'] = abs(result['Error (bar)'])
result['Percentage_Error (%)'] = abs(result['Error (bar)'] / (result[TARGET_COL] + 1e-8)) * 100

# ترتیب ستون‌ها
cols_order = ['RecordDate', 'RecordTime', 'id',
              TARGET_COL, f'{TARGET_COL}_Predicted',
              'Error (bar)', 'Absolute_Error (bar)', 'Percentage_Error (%)']
remaining = [c for c in result.columns if c not in cols_order]
result = result[cols_order + remaining]

# ذخیره
result.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')

print("\n" + "="*80)
print("             پیش‌بینی فشار پمپ اصلی روغن با موفقیت انجام شد!")
print("="*80)
print(f"   فایل خروجی → {OUTPUT_FILE}")
print(f"   تارگت        → {TARGET_COL} (فشار پمپ اصلی روغن)")
print(f"   MAE          → {mae:.6f} bar")
print(f"   MAPE         → {mape:.3f}%")
print(f"   R²           → {r2:.6f}")
print("="*80)

شروع پیش‌بینی فشار پمپ اصلی روغن → AssetID_8343
فایل خوانده شد → 3,000 ردیف
موفقیت: ستون AssetID_8343 پیدا شد (فشار پمپ اصلی روغن)
نتایج نهایی پیش‌بینی فشار پمپ اصلی روغن (AssetID_8343)
MAE   : 1.198087 bar
RMSE  : 3.411013 bar
R²    : 0.076711
MAPE  : 12.962%

             پیش‌بینی فشار پمپ اصلی روغن با موفقیت انجام شد!
   فایل خروجی → C:\BI\regression_output_for_assetid_8343.xlsx
   تارگت        → AssetID_8343 (فشار پمپ اصلی روغن)
   MAE          → 1.198087 bar
   MAPE         → 12.962%
   R²           → 0.076711
